In [22]:
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline
plt.rcParams['figure.facecolor'] = 'white'

In [23]:
X=[[0,0],[0,1],[1,0],[1,1]]
print(X)
X=np.reshape(X, (4, 2))
print(X)
Y=[0,1,1,0]
print(Y)
Y=np.reshape(Y, (4, 1))
print(Y)


[[0, 0], [0, 1], [1, 0], [1, 1]]
[[0 0]
 [0 1]
 [1 0]
 [1 1]]
[0, 1, 1, 0]
[[0]
 [1]
 [1]
 [0]]


In [24]:
def sig(z):
    return 1 / (1 + np.exp(-z))

def dsig_dz(z):
    return sig(z) * (1 - sig(z))

def J(y, yhat):
    return -(y - yhat)

def dJ_dy(y, yhat):
    return (-(y - yhat))

In [25]:
n_input = 2
n_hidden = 2
n_output = 1

In [26]:
w1 = [[0.4,0.6],[0.3,0.8]]
w1=np.reshape(w1, (2, 2))
w2 = [0.1,0.7]
w2=np.reshape(w2, (2, 1))
print("w1\n",w1)
print("w2\n",w2)

b1 = np.random.normal(0,0.1, size=(n_hidden, 1))
b2 = np.random.normal(0,0.1, size=(n_output, 1))
print("b1\n",b1)
print("b2\n",b2)

w1
 [[0.4 0.6]
 [0.3 0.8]]
w2
 [[0.1]
 [0.7]]
b1
 [[-0.055857  ]
 [-0.06263402]]
b2
 [[0.02521021]]


In [27]:
def forward1(x0, w1, b1, w2, b2):
    x1 = sig(np.dot(x0, w1) + b1.T)  # output of hidden layer
    return sig(np.dot(x1, w2) + b2.T)  # output of output layer

In [28]:
def forward2(x0, w1, w2):
    hls = np.dot(x0, w1)
    hla = sig(hls)  # output of hidden layer
    ols = np.dot(hla, w2)
    ola = sig(ols)
    return ola  # output of output layer

In [29]:
yhat = forward2(X, w1, w2)
yhat

array([[0.59868766],
       [0.6319116 ],
       [0.62523494],
       [0.65211467]])

In [30]:
def backward1(x0, w1, b1, w2, b2, y, yhat, alpha):
    # quantities
    z1 = np.dot(x0, w1) + b1.T
    x1 = sig(z1)
    z2 = np.dot(x1, w2) + b2.T
    y = sig(z2)

    delta2 = dJ_dy(y, yhat) * dsig_dz(z2)
    delta1 = np.matmul(w2, delta2) * dsig_dz(z1).T

    w2 -= alpha * np.multiply(delta2, x1).T
    w1 -= alpha * np.multiply(delta1, x0).T

    b2 -= alpha * delta2
    b1 -= alpha * delta1
    
    return w1, b1, w2, b2

In [31]:
def backward2(x0, w1,w2, y,alpha):
    # quantities
    z1 = np.dot(x0, w1)
    
    az1 = sig(z1)
    
    z2 = np.dot(az1, w2)
    
    yhat = sig(z2)
    yhat=np.reshape(yhat, (4, 1))
    
    
    djdy = dJ_dy(y, yhat)
    dsigdz =  dsig_dz(z2)   
 
    dsigdz=np.reshape(dsigdz, (4, 1))
    delta2 = djdy * dsigdz
 
    delta1 = np.multiply(delta2,az1)
    lndt1 = delta1.shape
    w2 = w2.T
    dw2 = w2.copy()
    for dj in range(4):
        dw2 -= alpha* delta1[dj, :]
    
    
    delta3 = np.matmul(delta2 , w2)
    
    hidden_act_derviative=(az1*(1-az1))

    pr = np.multiply(delta3,hidden_act_derviative)
    C = []
    for i in range(len(pr)):
        ls = []
        for j in range(len(pr[0])):
            for k in range(len(x0[0])):
                cal_value=pr[i][j]*x0[i][k]
                if(cal_value == 0):
                    cal_value = abs(cal_value)
                ls.append(cal_value)
        C.append(ls)

    for i in range(len(w1)):
        for j in range(len(C[0])):
            w1[i][0] = w1[i][0]-alpha*C[i][j]
    
    w2=dw2.reshape((2,1)) #dw2 is of shape (1,2) reshape it to (2,1)

    return w1, w2

In [32]:
def check_successive(prev_array, curr_array):
    diff = prev_array-curr_array
    return np.any(diff<0.001)

In [33]:
alpha=0.1
yhat = forward2(X, w1, w2)
epochs = 500

while epochs:
    prev_w1, prev_w2 = w1,w2
    w1, w2 = backward2(X, w1, w2, Y, alpha)
    epochs-=1
    #print("PREV",prev_w1,prev_w2)
    #print("NEW",w1,w2)
    if check_successive(prev_w1,w1) and check_successive(prev_w2,w2):
        print("Final Weights total trained in ",epochs,"epochs")
        print("W1=",w1)
        print("W2=",w2)
        
        break
else:
    print("Epochs Exhausted")


Final Weights total trained in  396 epochs
W1= [[0.4       0.6      ]
 [0.3746229 0.8      ]]
W2= [[-0.24596872]
 [ 0.34400101]]
